In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# from torch_geometric.nn import GCNConv

In [2]:
class GCNConvByHand(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [3]:
class GCNByHand(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels, in_channels)
        self.conv2 = GCNConvByHand(in_channels, in_channels)
        self.out = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        print("x after second layer\n", x)
        x = torch.relu(x)
        x = torch.transpose(x, 0, 1)
        x = self.out(x)
        print("x after output layer\n", x)
        x = F.softmax(x, dim=1)
        # x = torch.transpose(x, 0, 1)
        return x

In [ ]:
num_nodes = 3
num_features = 2
num_labels = 5

# the following needs to be replaced by some node embeddings
x = torch.randn((num_features, num_nodes)) # D x N

# print("x", x, x.shape)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(in_channels=num_features, out_channels=num_labels)

out = model(x, A)

print("output")
print(out)

x after first layer
 tensor([[ 1.3221,  1.4846,  0.9196],
        [-1.7021, -0.5658, -1.1658]], grad_fn=<AddBackward0>)
x after second layer
 tensor([[ 1.7493,  1.4104,  1.2021],
        [-2.6483, -2.0801, -1.7310]], grad_fn=<AddBackward0>)
x after output layer
 tensor([[ 4.1769e-01,  3.1422e-01, -7.6756e-05, -1.0466e+00, -4.9544e-01],
        [ 4.5486e-01,  2.7156e-01, -4.1740e-02, -9.3422e-01, -4.1416e-01],
        [ 4.7770e-01,  2.4535e-01, -6.7341e-02, -8.6519e-01, -3.6421e-01]],
       grad_fn=<AddmmBackward0>)
output
tensor([[0.3132, 0.2824, 0.2063, 0.0724, 0.1257],
        [0.3216, 0.2677, 0.1957, 0.0802, 0.1349],
        [0.3263, 0.2587, 0.1892, 0.0852, 0.1406]], grad_fn=<SoftmaxBackward0>)


In [5]:
# out[2, :].sum()

In [6]:
A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
preactivation

array([[ 1.32212353,  1.48463957,  0.91958335],
       [-1.70210862, -0.56578958, -1.16578603]])